# Kaggle Setup

In [1]:
import os

path = "/kaggle/working"
is_empty = (len(os.listdir(path)) == 1)

# !cp -r /kaggle/input/arageneval2025-task1-sc/* /kaggle/working/

if is_empty:
    print(f"✅ '{path}' is empty. copying project strcuture into it ...")
    !cp -r /kaggle/input/arageneval2025-task1-sc/* /kaggle/working/
    # !cp -r /kaggle/input/arageneval2025-task1-dataset/* /kaggle/working/
    # !mv /kaggle/working/data/data /kaggle/working/data_tmp
    # !rm -r /kaggle/working/data
    # !mv /kaggle/working/data_tmp /kaggle/working/data
else:
    print(f"❌ '{path}' is not empty — contains {len(os.listdir(path))} items.")




# Change to project directory
%cd /kaggle/working/ 

# Confirm
!ls

❌ '/kaggle/working' is not empty — contains 12 items.
/kaggle/working
config.py  evaluation	models	 __pycache__	  sc_results.zip  utils
data	   __init__.py	prompts  run_pipeline.py  state.db


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# # Navigate to project folder
# %cd /content/drive/MyDrive/AraGenEval2025
# !ls

In [4]:
# Add project root to Python path
import os
import sys
sys.path.insert(0, os.getcwd())
print("✅ Project root added to sys.path:", os.getcwd())

✅ Project root added to sys.path: /kaggle/working


# Dependencies

In [5]:
!pip install --upgrade transformers==4.41.2 datasets==2.19.1 scikit-learn torch wandb python-dotenv evaluate --quiet
!pip install -U transformers==4.41.2 peft==0.11.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 84.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 105.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 110.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 79.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 8.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
# import os
# os.kill(os.getpid(), 9)

In [7]:
import torch
from datasets import Dataset
import evaluate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score  
import pandas as pd
import numpy as np


2025-07-07 09:04:22.570662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751879062.769367      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751879062.823190      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
# Update these paths with your actual dataset locations on Drive
train_dataset_path ='data/AuthorshipStyleTransferTrain.xlsx'
val_dataset_path =  'data/AuthorshipStyleTransferVal.xlsx'

In [9]:
import sys
import os

project_root = os.getcwd()  # Should be /content/drive/MyDrive/AraGenEval2025
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root added to sys.path: {project_root}")

Project root added to sys.path: /kaggle/working


In [10]:
!pwd

/kaggle/working


In [11]:
from config import Config

print(Config.TRAIN_FILE)

data/AuthorshipStyleTransferTrain.xlsx


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
import gc

def load_and_sample_dataset(
    file_path,
    sample_mode="all",    # Options: "all", "random", "stratified"
    sample_size=None,     # For "random"/"stratified": float (proportion) or int (absolute count)
    random_state=42
):
    """
    Load dataset from Excel and apply sampling.

    Args:
        file_path (str): Path to dataset file.
        sample_mode (str): "all", "random", or "stratified".
        sample_size (float|int): Proportion (0-1) or count for sampling.
        random_state (int): Random seed for reproducibility.

    Returns:
        pd.DataFrame: Sampled dataset.
    """
    print(f"📂 Loading dataset from: {file_path}")
    df = pd.read_excel(file_path, engine='openpyxl')

    if sample_mode == "all":
        sampled_df = df.copy()
        print(f"✅ Loaded full dataset with {len(sampled_df)} samples.")

    elif sample_mode == "random":
        if sample_size is None:
            raise ValueError("For 'random' mode, SAMPLE_SIZE must be set.")
        sampled_df = df.sample(
            n=sample_size if isinstance(sample_size, int) else int(len(df) * sample_size),
            random_state=random_state
        )
        print(f"✅ Randomly sampled {len(sampled_df)} samples.")

    elif sample_mode == "stratified":
        if sample_size is None or not (0 < sample_size < 1):
            raise ValueError("For 'stratified' mode, SAMPLE_SIZE must be a proportion between 0 and 1.")
        sampled_df, _ = train_test_split(
            df,
            train_size=sample_size,
            stratify=df['author'],
            random_state=random_state
        )
        print(f"✅ Stratified sampled {len(sampled_df)} samples (author distribution preserved).")

    else:
        raise ValueError(f"Invalid sample_mode: {sample_mode}")

    del df
    gc.collect()

    return sampled_df


In [14]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# model_name = "megabot131/m-e5-large-toxic-classification-lora"   # "microsoft/Multilingual-MiniLM-L12-H384" #"CAMeL-Lab/bert-base-arabic-camelbert-ca" "aubmindlab/aragpt2-mega-detector-long"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# print("Max tokens:", tokenizer.model_max_length)

# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     # num_labels=num_labels
#     # remove use_safetensors=True
# )
# print("Model context limit:", model.config.max_position_embeddings)

# Load Dataset

In [15]:
# === CONFIG ===
SAMPLE_MODE = "stratified"  # "all", "random", "stratified"
SAMPLE_SIZE = 0.1           # 10% for stratified or random. Set to None for full dataset.

# === LOAD AND SAMPLE ===
train_df = load_and_sample_dataset(
    Config.TRAIN_FILE,
    # sample_mode=SAMPLE_MODE,
    # sample_size=SAMPLE_SIZE
)

# === LOAD AND SAMPLE ===
val_df = load_and_sample_dataset(
    Config.VAL_FILE,
    # sample_mode=SAMPLE_MODE,
    # sample_size=SAMPLE_SIZE
)

print("Train shape:", train_df.shape)
print("Validation shape:", val_df.shape)
print("Authors in Train:", train_df['author'].nunique())
print("Authors in Val:", val_df['author'].nunique())

# train_df.head()

📂 Loading dataset from: data/AuthorshipStyleTransferTrain.xlsx
✅ Loaded full dataset with 35122 samples.
📂 Loading dataset from: data/AuthorshipStyleTransferVal.xlsx
✅ Loaded full dataset with 4157 samples.
Train shape: (35122, 4)
Validation shape: (4157, 4)
Authors in Train: 21
Authors in Val: 21


In [16]:
train_data = [{"text": row["text_in_author_style"], "label": row["author"]} for _, row in train_df.iterrows()]
val_data = [{"text": row["text_in_author_style"], "label": row["author"]} for _, row in val_df.iterrows()]

unique_authors = sorted(set(train_df["author"]).union(set(val_df["author"])))
author2id = {author: idx for idx, author in enumerate(unique_authors)}

for item in train_data:
    item["label"] = author2id[item["label"]]
for item in val_data:
    item["label"] = author2id[item["label"]]

train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

In [17]:
import json
with open("/kaggle/working/evaluation/ar_style_classifier/results/author2id.json", "w") as f:
    json.dump(author2id, f, indent=4)

# Tokenization

In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "CAMeL-Lab/bert-base-arabic-camelbert-ca" #"allenai/longformer-base-4096" #"UBC-NLP/AraT5v2-base-1024"
num_labels = len(unique_authors)
tokenizer = AutoTokenizer.from_pretrained(model_name,use_safetensors=True)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    # use_safetensors=True
).to(device)

def preprocess_sliding(examples):
    chunk_size = 512
    stride = 256
    input_ids_list = []
    attention_mask_list = []
    label_list = []
    sample_id_list = []  # NEW

    for idx, (text, label) in enumerate(zip(examples["text"], examples["label"])):
        encodings = tokenizer(
            text,
            truncation=True,
            max_length=chunk_size,
            stride=stride,
            return_overflowing_tokens=True,
            return_attention_mask=True,
        )
        
        num_chunks = len(encodings["input_ids"])
        for input_ids, attention_mask in zip(encodings["input_ids"], encodings["attention_mask"]):
            input_ids_list.append(input_ids)
            attention_mask_list.append(attention_mask)
            label_list.append(label)
            sample_id_list.append(idx)  # Use the index as sample identifier

    return {
        "input_ids": input_ids_list,
        "attention_mask": attention_mask_list,
        "label": label_list,
        "sample_id": sample_id_list
    }


train_dataset = train_dataset.map(preprocess_sliding, batched=True, remove_columns=["text"])
val_dataset = val_dataset.map(preprocess_sliding, batched=True, remove_columns=["text"])

tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-ca and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/35122 [00:00<?, ? examples/s]

Map:   0%|          | 0/4157 [00:00<?, ? examples/s]

In [16]:
print(train_dataset[0])


{'label': 20, 'input_ids': [2, 2159, 5018, 2061, 1, 2837, 6201, 9887, 3153, 6, 1908, 3430, 3368, 4307, 21673, 6, 1912, 25237, 2038, 9993, 2731, 378, 1, 1, 1, 3720, 6045, 7475, 20317, 4335, 13987, 1, 1963, 7388, 2045, 2217, 1, 378, 2086, 1, 1, 26032, 1007, 6270, 1912, 2038, 3192, 30, 4910, 1, 1, 1908, 7933, 1014, 23613, 1013, 378, 1, 1, 2278, 7638, 1043, 29899, 1043, 380, 4053, 7110, 13991, 17, 5708, 2722, 7397, 2075, 20642, 2525, 4697, 6822, 24762, 7811, 7475, 16491, 12045, 1912, 7855, 378, 20471, 1, 2395, 2905, 9446, 12, 9377, 1, 1, 1, 1963, 15127, 8244, 7192, 26468, 1007, 13, 378, 1, 4423, 4884, 1979, 1, 8513, 1028, 1912, 2085, 6067, 6900, 378, 1, 378, 2086, 1, 1, 2794, 20585, 1028, 2743, 2734, 2379, 18, 28918, 13498, 1, 1972, 1, 23246, 6, 7931, 3552, 6, 378, 5632, 1911, 1, 2339, 12250, 8085, 1007, 17488, 10244, 18647, 378, 2073, 2103, 1, 6064, 11059, 2669, 378, 1, 11113, 3721, 1, 2574, 378, 2637, 1963, 16363, 1, 2813, 18, 1, 378, 2427, 23350, 2038, 3666, 1016, 4759, 11462, 2083, 378

## Evaluation

In [61]:
# # Load metrics
# metric_acc = evaluate.load("accuracy")
# metric_f1 = evaluate.load("f1")


# def compute_metrics(p):
#     preds = np.argmax(p.predictions, axis=1)
#     acc = metric_acc.compute(predictions=preds, references=p.label_ids)["accuracy"]
#     f1 = metric_f1.compute(predictions=preds, references=p.label_ids)["f1"]
#     return {"accuracy": acc, "f1": f1}



import numpy as np
import evaluate

# Load metrics
metric_acc = evaluate.load("accuracy")
metric_f1 = evaluate.load("f1")

# Sanity check for labels
EXPECTED_NUM_CLASSES = len(unique_authors)  # make sure this is defined correctly


def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)

    # Defensive check: Are labels within expected range?
    if np.max(preds) >= EXPECTED_NUM_CLASSES or np.min(preds) < 0:
        raise ValueError(f"Predictions contain invalid class indices: {np.unique(preds)}")

    if np.max(p.label_ids) >= EXPECTED_NUM_CLASSES or np.min(p.label_ids) < 0:
        raise ValueError(f"Label IDs contain invalid class indices: {np.unique(p.label_ids)}")

    acc = metric_acc.compute(predictions=preds, references=p.label_ids)["accuracy"]
    
    # Use macro average for multiclass tasks
    f1 = metric_f1.compute(predictions=preds, references=p.label_ids, average="macro")["f1"]
    
    return {"accuracy": acc, "f1": f1}



## Training Setup

In [62]:
# Dummy sanity check
dummy_preds = np.random.randint(0, EXPECTED_NUM_CLASSES, size=100)
dummy_labels = np.random.randint(0, EXPECTED_NUM_CLASSES, size=100)

test_acc = metric_acc.compute(predictions=dummy_preds, references=dummy_labels)["accuracy"]
test_f1 = metric_f1.compute(predictions=dummy_preds, references=dummy_labels, average="macro")["f1"]

print(f"Sanity metric test - Accuracy: {test_acc:.4f}, F1 (macro): {test_f1:.4f}")


Sanity metric test - Accuracy: 0.0400, F1 (macro): 0.0351


In [63]:
print(len(train_dataset))

43507


In [64]:
import os
os.cpu_count()

4

In [71]:
from transformers import Trainer, TrainingArguments

## batch size calculation
effective_batch_size = 32
batch_size = 16
accumulation_steps = effective_batch_size // batch_size
epochs = 5
OUTPUT_DIR="evaluation/ar_style_classifier/"

#config training env
model.gradient_checkpointing_disable()
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# os.environ["WANDB_DISABLED"] = "true"



# args
training_args = TrainingArguments(
    output_dir="evaluation/ar_style_classifier/",
    max_steps=-1,
    eval_strategy="epoch",
    save_strategy="epoch",
    # save_steps=500,
    # eval_steps=1000,
    # eval_strategy="steps",
    # save_strategy="steps",
    # save_steps=10,
    # eval_steps=10,  # ⬅️ super frequent eval for debugging
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    gradient_accumulation_steps=accumulation_steps,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=2,
    logging_steps=accumulation_steps,
    logging_strategy="steps",
    dataloader_num_workers=0,
    report_to="none"
)

In [72]:
from transformers import DataCollatorForSeq2Seq
from transformers import Trainer, TrainingArguments, DefaultDataCollator

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    callbacks=[],  # Completely remove default callbacks like WandB
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Eval dataset size: {len(val_dataset)}")

Train dataset size: 43507
Eval dataset size: 5031


In [73]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

## Launch Training

In [ ]:
from datetime import datetime
import time
import os
from config import Config


print(f"📢 Starting training at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

start_time = time.time()

# Detect checkpoint
last_checkpoint = None
CHECKPOINT_DIR = OUTPUT_DIR    #os.path.join(Config.MODEL_WEIGHTS_FOLDER,safe_model_name(MODEL_CONFIG["name"]))
RESUME_TRAINING=False

if RESUME_TRAINING:
    if os.path.isdir(CHECKPOINT_DIR):
        checkpoints = [d for d in os.listdir(CHECKPOINT_DIR) if d.startswith("checkpoint-")]

        if checkpoints:
            # Sort numerically based on step number
            latest_checkpoint =  max(checkpoints, key=lambda x: int(x.split("-")[-1]))
            last_checkpoint = os.path.join(CHECKPOINT_DIR, latest_checkpoint)
            print(f"✅ Found checkpoint at {last_checkpoint}. Will resume training from there.")
        else:
            print("⚠️ No checkpoints found. Starting from scratch.")
    else:
        print("⚠️ Checkpoint directory doesn't exist. Starting from scratch.")
else:
    print("🆕 Starting fresh training (no checkpoint resume).")

# Start training
try:
    if last_checkpoint:
        train_output = trainer.train(
            resume_from_checkpoint=last_checkpoint,
            # ignore_keys_for_eval=["optimizer", "scheduler"]
        )
    else:
        train_output = trainer.train()

    total_time = time.time() - start_time
    print(f"✅ Training completed in {total_time/3600:.2f} hours")

    if train_output.metrics:
        print("\n📊 Final Training Metrics:")
        for key, value in train_output.metrics.items():
            print(f" {key}: {value:.4f}")

except KeyboardInterrupt:
    print("\n⚠️ Training interrupted by user. Saving checkpoint...")
    interrupted_path = os.path.join(CHECKPOINT_DIR, "checkpoint-interrupted")
    trainer.save_model(interrupted_path)
    print(f"💾 Checkpoint saved at {interrupted_path}")

📢 Starting training at 2025-07-02 13:24:00
🆕 Starting fresh training (no checkpoint resume).


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.009800,0.775574,0.832240,0.789812
2,0.000700,0.693766,0.861459,0.813936
3,0.047600,0.820671,0.857682,0.797888


## Save Results

In [ ]:
output_dir = "evaluation/ar_style_classifier/results"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model saved to {output_dir}")

## Inference

In [51]:
# # --- Inference on full validation texts ---
# def predict_full_text(text):
#     enc = tokenizer(
#         text,
#         truncation=True,
#         padding="max_length",
#         max_length=512,
#         stride=256,
#         return_overflowing_tokens=True,
#         return_tensors="pt"
#     ).to(device)
#     logits = model(**enc).logits  # [num_chunks, num_labels]
#     avg_logits = logits.mean(dim=0)
#     probs = torch.softmax(avg_logits, dim=0)
#     return torch.argmax(probs).item(), probs.cpu().tolist()

# # --- Evaluate across validation set ---
# preds, refs = [], []
# for example in val_df.itertuples():
#     pred, _ = predict_full_text(example.text_in_author_style)
#     preds.append(pred)
#     refs.append(author2id[example.author])

# from sklearn.metrics import accuracy_score, f1_score
# print("Validation Accuracy:", accuracy_score(refs, preds))
# print("Validation F1‑macro:", f1_score(refs, preds, average="macro"))

